<a href="https://colab.research.google.com/github/sunil-karki/NLP-Question_Classification/blob/master/RealorNot%3FNLP_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#The learning rate changes with every iteration, i.e., with every batch and not epoch. So, if you set the decay = 1e-2 and each epoch has 100 batches/iterations,
#then after 1 epoch your learning rate will be

#  lr = init_lr * 1/(1 + 1e-2 * 100)
#So, if I want my learning rate to be 0.75 of the original learning rate at the end of each epoch, I would set the lr_decay to

#  batches_per_epoch = dataset_size/batch_size
#  lr_decay = (1./0.75 -1)/batches_per_epoch
#It seems to work for me. Also, since the new learning rate is calculated at every iteration, the optimizer doesn't change the value of the learning rate 
#variable and always uses the initial learning rate to calculate the effective learning rate.


In [0]:
#Pyimage decay
#https://www.pyimagesearch.com/2019/07/22/keras-learning-rate-schedules-and-decay/

In [2]:
#https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 19.8MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 56.7MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 59.4MB/s]


In [5]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate

from keras.optimizers import Adam
from keras import backend as K      ######
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [7]:

train_df = pd.read_csv('train.csv') 
test_df = pd.read_csv('test.csv') 
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (7613, 5)
Test shape :  (3263, 4)


In [8]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [9]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [0]:

train_data = train_df 
train_data = train_data.replace(np.nan, '', regex=True)

train_data['text'] = train_data['text'] + ' ' + train_data['location'] + ' ' + train_data['keyword']
del train_data['keyword']
del train_data['location']
#train_data.iloc[0]
a = train_data.iloc[0][['text']].values
#print(a)

test_data = test_df 
test_data = test_data.replace(np.nan, '', regex=True)

test_data['text'] = test_data['text'] + ' ' + test_data['location'] + ' ' + test_data['keyword']

In [11]:
train_data

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [12]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#train_data['tweet'].apply(lambda x: [item for item in x if item not in stop])

import nltk
import re
from nltk.corpus import stopwords
stopword = stopwords.words('english')


lm = nltk.WordNetLemmatizer()
st = nltk.PorterStemmer()

class NLP_preprocess:
    def tokenize(self, text):
        tokens = re.split('\W+', text)
        return tokens

    def remove_stopwords(self, tokenized_text):
        text = [word for word in tokenized_text if word not in stopword]
        return text
    
    def lemmatizing(self, text):
        text = [lm.lemmatize(word) for word in text]
        return text
    
    def stemming(self, text):
        text = [st.stem(word) for word in text]
        return text

    def process_column(self, col):
        col = col.apply(lambda x: self.tokenize(x.lower()))
        col = col.apply(lambda x: self.remove_stopwords(x))
        col = col.apply(lambda x: self.lemmatizing(x))
        return col

In [14]:
#Tokenizing the text and removing stop words
nlp = NLP_preprocess()

train_data['text'] = nlp.process_column(train_data['text'])
train_data

,id,text,target
0,1,"[deed, reason, earthquake, may, allah, forgive...",1
1,4,"[forest, fire, near, la, ronge, sask, canada, ]",1
2,5,"[resident, asked, shelter, place, notified, of...",1
3,6,"[13, 000, people, receive, wildfire, evacuatio...",1
4,7,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1
...,...,...,...
7608,10869,"[two, giant, crane, holding, bridge, collapse,...",1
7609,10870,"[, aria_ahrary, thetawniest, control, wild, fi...",1
7610,10871,"[m1, 94, 01, 04, utc, 5km, volcano, hawaii, ht...",1
7611,10872,"[police, investigating, e, bike, collided, car...",1


In [15]:
from nltk.tokenize.treebank import TreebankWordDetokenizer as detoken

#Detokenizing the token back to text
#TreebankWordDetokenizer().detokenize(train_data.iloc[0]['text'])

train_data['text'] = train_data['text'].apply(lambda x: detoken().detokenize(x))
train_data

,id,text,target
0,1,deed reason earthquake may allah forgive u,1
1,4,forest fire near la ronge sask canada,1
2,5,resident asked shelter place notified officer ...,1
3,6,13 000 people receive wildfire evacuation orde...,1
4,7,got sent photo ruby alaska smoke wildfire pour...,1
...,...,...,...
7608,10869,two giant crane holding bridge collapse nearby...,1
7609,10870,aria_ahrary thetawniest control wild fire cali...,1
7610,10871,m1 94 01 04 utc 5km volcano hawaii http co zdt...,1
7611,10872,police investigating e bike collided car littl...,1


In [0]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

## ValueError: Layer weight shape (95000, 300) not compatible with provided weight shape (24568, 300)
max_features = 24568
max_features = 13308

In [17]:
maxlen = max([len(data) for data in train_data['text'].values])   ##maximum number of words in column text
print(maxlen)    ##195

180


In [0]:
## split to train and val
train_data, val_data = train_test_split(train_data, test_size=0.15, random_state=2018)
train_data, traintest_data = train_test_split(train_data, test_size=0.15, random_state=2019) 
 

In [19]:
train_data

,id,text,target
4345,6170,latestnews tension bayelsa patience jonathan p...,1
5917,8448,godof_mischief_ loki dagger pulled jammed mina...,1
6636,9503,terrorist tried get car shot http co vsoxkbt6n...,1
4277,6075,greatbritishbakeoff love know nice weather hap...,0
2636,3780,day 1998 appetite destruction go 1 billboard a...,0
...,...,...,...
7126,10208,jetstar virgin forced cancel bali flight ash m...,1
5847,8355,understand wanting hang guy friend give space ...,0
6138,8757,nowplaying song siren john frusciante 2009 htt...,0
6876,9859,back ireland v sad traumatised freezing beauti...,0


In [20]:
print(type(train_data))
print(train_data.shape)
train_data = train_data.sample(frac=1)

<class 'pandas.core.frame.DataFrame'>
(5500, 3)


In [21]:
print(train_data)

         id                                               text  target
5716   8158  video picking body water rescuer searching hun...       1
3120   4480  also iphone charger broken electrocuted newcas...       0
2288   3283  read already 14 remains one favorite article e...       0
5411   7722   panicking made tired __ want sleep bed panicking       1
6290   8986  last two weather pic storm august 2nd people p...       1
...     ...                                                ...     ...
5320   7595       tube strike absolute pandemonium pandemonium       0
2429   3490  joelsherman1 dw way better career chipper dera...       0
859    1241  setzorah dad dont claim mean right look eye bl...       0
7597  10852  mh370 aircraft debris found la reunion missing...       1
3752   5331  politifiact harry reid 30 percent woman served...       0

[5500 rows x 3 columns]


In [0]:
train_data = train_data.sample(frac=1)
train_data = pd.concat([train_data, train_data.iloc[0:200]]) 
train_data = train_data.sample(frac=1)
train_data = pd.concat([train_data, train_data.iloc[0:200]]) 
train_data = train_data.sample(frac=1)
train_data = pd.concat([train_data, train_data.iloc[0:200]]) 
train_data = train_data.sample(frac=1)
train_data = pd.concat([train_data, train_data.iloc[0:200]]) 
train_data = train_data.sample(frac=1)
train_data = pd.concat([train_data, train_data.iloc[0:200]]) 
train_data = train_data.sample(frac=1)
train_data = pd.concat([train_data, train_data.iloc[0:200]]) 

In [23]:
print(train_data)

        id                                               text  target
4335  6156  criminal hijack lorry bus arrested enugu accor...       1
6407  9159  17 killed ûªarabia mosque suicide bombing suic...       1
6943  9958                              feel lucky rn tsunami       0
3764  5346  fire catalinas look kinda cool picture justice...       1
1219  1757  zourryart forgot add burning building screamin...       1
...    ...                                                ...     ...
2882  4142  happily support mydrought project bringing awa...       1
6314  9026               put stretcher want see dmv stretcher       0
2539  3643    np agalloch desolation song istanbul desolation       0
1555  2245  laevantine fortunately reworked plumbing emerg...       0
331    478  erker eep thought yesterday saw hella scary ha...       1

[6700 rows x 3 columns]


In [0]:

## fill up the missing values
train_X = train_data['text'].values
val_X = val_data['text'].values
test_X = test_data['text'].values
traintest_X = traintest_data['text'].values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)
traintest_X = tokenizer.texts_to_sequences(traintest_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)
traintest_X = pad_sequences(traintest_X, maxlen=maxlen)

## Get the target values
train_y = train_data['target'].values   #train_y is an array i.e. array([1, 0, 1, ..., 1, 1, 1])
val_y = val_data['target'].values
traintest_y = traintest_data['target'].values  
    
#shuffling the data
np.random.seed(2021)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))
trntst_idx = np.random.permutation(len(traintest_X))

train_X = train_X[trn_idx]  #train_X is an array i,e. array([1653, 2280, 1170, ..., 6370, 2662, 1274])
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]    
traintest_X = traintest_X[trntst_idx] 
traintest_y = traintest_y[trntst_idx]

In [25]:
word_index = tokenizer.word_index
print(word_index)

{'co': 1, 'http': 2, 'û': 3, 'new': 4, 'fire': 5, 'like': 6, 'amp': 7, 'u': 8, 'body': 9, 'emergency': 10, 'usa': 11, 'get': 12, '2': 13, 'building': 14, 'via': 15, 'burning': 16, 'california': 17, 'one': 18, 'nuclear': 19, 'people': 20, 'police': 21, 'disaster': 22, 'news': 23, 'suicide': 24, 'city': 25, 'death': 26, 'ca': 27, 'video': 28, 'storm': 29, 'flood': 30, 'world': 31, 'york': 32, 'day': 33, 'hostage': 34, 'time': 35, 'year': 36, 'state': 37, 'bomb': 38, 'weapon': 39, 'attack': 40, 'injury': 41, 'fatality': 42, 'back': 43, '3': 44, 'would': 45, 'crash': 46, 'home': 47, 'collapse': 48, 'family': 49, 'still': 50, 'mass': 51, 'first': 52, '1': 53, 'siren': 54, 'go': 55, 'london': 56, 'say': 57, 'got': 58, 'know': 59, 'life': 60, 'gt': 61, 'dead': 62, 'wildfire': 63, 'forest': 64, '5': 65, 'casualty': 66, 'uk': 67, 'see': 68, 'man': 69, 'accident': 70, 'united': 71, '4': 72, 'look': 73, 'earthquake': 74, 'let': 75, 'rt': 76, 'bombing': 77, 'love': 78, 'car': 79, 'bag': 80, 'train

In [26]:
train_X

array([[    0,     0,     0, ...,     2,     1,  3864],
       [    0,     0,     0, ...,  6100,   717,   411],
       [    0,     0,     0, ...,  4211,   541,   171],
       ...,
       [    0,     0,     0, ...,   273,   293,   279],
       [    0,     0,     0, ..., 12753,   577,   320],
       [    0,     0,     0, ...,  9301,    67,   284]], dtype=int32)

In [27]:
!kaggle competitions download -c quora-insincere-questions-classification

100% 5.95G/5.96G [01:26<00:00, 18.2MB/s]
100% 5.96G/5.96G [01:27<00:00, 73.4MB/s]
 75% 41.0M/54.9M [00:01<00:00, 24.6MB/s]
100% 54.9M/54.9M [00:01<00:00, 49.4MB/s]
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 66.9MB/s]
 32% 5.00M/15.8M [00:00<00:00, 27.0MB/s]
100% 15.8M/15.8M [00:00<00:00, 62.7MB/s]


In [28]:
!ls

embeddings.zip	sample_submission.csv	   test.csv.zip
kaggle.json	sample_submission.csv.zip  train.csv
sample_data	test.csv		   train.csv.zip


In [29]:
!unzip embeddings.zip

Archive:  embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.840B.300d/glove.840B.300d.txt  
  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  


In [30]:
!ls

embeddings.zip			sample_data		   train.csv
glove.840B.300d			sample_submission.csv	   train.csv.zip
GoogleNews-vectors-negative300	sample_submission.csv.zip  wiki-news-300d-1M
kaggle.json			test.csv
paragram_300_sl999		test.csv.zip


In [0]:
def load_glove(word_index):
    # EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    EMBEDDING_FILE = 'glove.840B.300d/glove.840B.300d.txt'
    ## This file contains content as following
    #Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223  .....
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [32]:
embedding_matrix_1 = load_glove(word_index)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [34]:
dataset_size = train_data.shape[0]
print(dataset_size)

6700


In [35]:
#batches_per_epoch = dataset_size/batch_size
learningrate = 0.15
batches_per_epoch = int(dataset_size/500)
lrdecay = (1./learningrate -1)/batches_per_epoch
print(lrdecay)

0.4358974358974359


In [0]:
from keras.optimizers import Adam, Adagrad


def model_gru_atten_3(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    #x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    
    #x = Bidirectional(GRU(128, return_sequences=True))(x)
    #x = Bidirectional(GRU(100, return_sequences=True))(x)
    #x = Bidirectional(GRU(64, return_sequences=True))(x)
    
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Dropout(0.7)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Dropout(0.7)(x)
    #x = Bidirectional(GRU(64, return_sequences=True))(x)
    
    x = Attention(maxlen)(x)        ## Finding on this
    #z = Dropout(0.1)(z)
    
    x = Dense(1, activation="sigmoid")(x)
    #x = Dense(1, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    #opt = Adagrad(lr=0.001, momentum=0.9, decay=0.01)
    opt = Adam(lr= learningrate, decay= lrdecay)
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [0]:
from keras.models import load_model

def train_pred(model, epochs_param):
    try:
        model.fit(train_X, train_y, batch_size=500, epochs = epochs_param, validation_data=(val_X, val_y))  #512
    except KeyboardInterrupt:
        filename = 'google3qla.h5'
        model.save(filename)
        model = load_model('google3qla.h5')

    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
    return pred_val_y, pred_test_y, model

In [38]:
'''
from keras.models import load_model

def train_pred(model, epochs_param):
    try:
        history = model.fit(train_X, train_y, batch_size=500, epochs = epochs_param, validation_data=(val_X, val_y), shuffle=True)  #512
    except KeyboardInterrupt:
        filename = 'google3qla.h5'
        model.save(filename)
        model = load_model('google3qla.h5')

    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
    return pred_val_y, pred_test_y, history, model
'''

"\nfrom keras.models import load_model\n\ndef train_pred(model, epochs_param):\n    try:\n        history = model.fit(train_X, train_y, batch_size=500, epochs = epochs_param, validation_data=(val_X, val_y), shuffle=True)  #512\n    except KeyboardInterrupt:\n        filename = 'google3qla.h5'\n        model.save(filename)\n        model = load_model('google3qla.h5')\n\n    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)\n    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)\n    return pred_val_y, pred_test_y, history, model\n"

In [0]:
'''
from keras.models import load_model
#works

def train_pred(model, epochs_param):
    history = model.fit(train_X, train_y, batch_size=500, epochs = epochs_param, validation_data=(val_X, val_y))  #512
    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
    return pred_val_y, pred_test_y, history, model
'''

'\nfrom keras.models import load_model\n#works\n\ndef train_pred(model, epochs_param):\n    history = model.fit(train_X, train_y, batch_size=500, epochs = epochs_param, validation_data=(val_X, val_y))  #512\n    pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)\n    pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)\n    return pred_val_y, pred_test_y, history, model\n'

In [39]:
pred_val_y, pred_test_y, model = train_pred(model_gru_atten_3(embedding_matrix_1), epochs_param = 50)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 6700 samples, validate on 1142 samples
Epoch 1/50
6700/6700 [==============================] - 79s 12ms/step - loss: 1.6901 - acc: 0.5272 - val_loss: 0.6970 - val_acc: 0.5762
Epoch 2/50
6700/6700 [==============================] - 69s 10ms/step - loss: 0.6664 - acc: 0.6019 - val_loss: 0.6385 - val_acc: 0.6103
Epoch 3/50
6700/6700 [==============================] - 68s 10ms/step - loss: 0.6391 - acc: 0.6324 - val_loss: 0.6248 - val_acc: 0.6410
Epoch 4/50
6700/6700 [==============================] - 69s 10ms/step - loss: 0.6203 - acc: 0.6437 - val_loss: 0.6127 - val_acc: 0.6436
Epoch 5/50
6700/6700 [==============================] - 69s 10ms/step - loss: 0.5969 - acc: 0.6724 - val_loss: 0.5944 - val_acc: 0.6673
Epoch 6/50
6700/6700 [===========================

In [0]:
import matplotlib.pyplot as plt

print(history.history.keys())
print("batch_size : ", 500, ", epoch : ", 30, ", loss= binary_crossentropy, optimizer= adam")
print("Bidirectional(GRU(100, return_sequences=True))(x)  x = Dropout(0.5)(x)")
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [0]:
##  ValueError: Layer weight shape (95000, 300) not compatible with provided weight shape (24568, 300)

In [40]:
score = model.evaluate(traintest_X, traintest_y, verbose=0) #79.48
print('test loss, test acc:', score)

test loss, test acc: [0.6755885462191275, 0.7950566428819965]


In [41]:
pred_test_y_a = np.where( pred_test_y > 0.42 , 1, 0)

print(pred_test_y_a.shape)

(3263, 1)


In [0]:
test_df = pd.read_csv("test.csv", usecols=["id"])
out_df = pd.DataFrame({"id":test_df["id"].values})
out_df['target'] = pred_test_y_a
out_df.to_csv("submission.csv", index=False)

In [43]:
!ls

embeddings.zip			sample_data		   test.csv.zip
glove.840B.300d			sample_submission.csv	   train.csv
GoogleNews-vectors-negative300	sample_submission.csv.zip  train.csv.zip
kaggle.json			submission.csv		   wiki-news-300d-1M
paragram_300_sl999		test.csv


In [44]:
sub = pd.read_csv("submission.csv")
sub

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [45]:
# submit the file to kaggle
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "Submitting Submission file"

100% 22.2k/22.2k [00:01<00:00, 12.2kB/s]
Successfully submitted to Real or Not? NLP with Disaster Tweets

In [0]:
pred_val_y, pred_test_y, model = train_pred(model_gru_atten_3(embedding_matrix_1), epochs_param = 5)

Train on 1994 samples, validate on 624 samples
Epoch 1/5
1994/1994 [==============================] - 40s 20ms/step - loss: 0.9337 - acc: 0.4860 - val_loss: 0.7651 - val_acc: 0.5929
Epoch 2/5
1994/1994 [==============================] - 29s 15ms/step - loss: 0.7142 - acc: 0.5682 - val_loss: 0.7080 - val_acc: 0.4071
Epoch 3/5
1994/1994 [==============================] - 29s 15ms/step - loss: 0.6967 - acc: 0.4824 - val_loss: 0.6729 - val_acc: 0.5785
Epoch 4/5
1994/1994 [==============================] - 29s 15ms/step - loss: 0.6716 - acc: 0.5727 - val_loss: 0.6774 - val_acc: 0.5929
Epoch 5/5
1994/1994 [==============================] - 29s 14ms/step - loss: 0.6769 - acc: 0.5682 - val_loss: 0.6557 - val_acc: 0.5962


ValueError: ignored

In [0]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [0]:
model = model_gru_atten_3(embedding_matrix_1)
history = model.fit(train_X, train_y, batch_size=500, epochs = 2, validation_data=(val_X, val_y))  #512

Train on 1994 samples, validate on 624 samples
Epoch 1/2
1994/1994 [==============================] - 40s 20ms/step - loss: 0.8320 - acc: 0.5371 - val_loss: 0.7975 - val_acc: 0.4071
Epoch 2/2
1994/1994 [==============================] - 29s 14ms/step - loss: 0.7230 - acc: 0.4669 - val_loss: 0.7033 - val_acc: 0.5929


In [0]:
pred_test_y

array([[0.01398143],
       [0.08510914],
       [0.96508956],
       ...,
       [0.98746103],
       [0.9870027 ],
       [0.00680384]], dtype=float32)

In [0]:
!ls

embeddings.zip			paragram_300_sl999	   test.csv
glove.840B.300d			sample_data		   test.csv.zip
google3qla.h5			sample_submission.csv	   train.csv
GoogleNews-vectors-negative300	sample_submission.csv.zip  train.csv.zip
kaggle.json			submission.csv		   wiki-news-300d-1M


In [0]:
sub = pd.read_csv("submission.csv")
sub

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [0]:
# submit the file to kaggle
# !kaggle competitions submit -c nlp-getting-started -f submission.csv -m "Submitting file through Google Colab!"

100% 22.2k/22.2k [00:00<00:00, 85.6kB/s]
Successfully submitted to Real or Not? NLP with Disaster Tweets

In [46]:
print(pred_val_y.shape)
print(test_X.shape)

(1142, 1)
(3263, 180)
